In [7]:
import caffe
from caffe.proto import caffe_pb2
from caffe import layers as L
from caffe import params as P
import random
import os

ImportError: cannot import name __version__

In [55]:
GPUId = 1 # can change this to 0,1,2
caffe.set_mode_gpu()
caffe.set_device(GPUId)

In [56]:
train_file_path = "data/development_kit/data/train.txt"
val_file_path = "data/development_kit/data/val.txt"
test_file_path = "data/development_kit/data/test.txt"

print "Training File Contents..."
content = [line.rstrip() for line in open(train_file_path)]
random.shuffle(content)
for x in content[:10]:
    print x

Training File Contents...
train/a/abbey/00000966.jpg 0
train/m/market/outdoor/00000553.jpg 65
train/c/clothing_store/00000086.jpg 34
train/c/candy_store/00000950.jpg 28
train/s/ski_slope/00000361.jpg 84
train/c/courtyard/00000730.jpg 42
train/b/botanical_garden/00000121.jpg 21
train/t/track/outdoor/00000331.jpg 95
train/h/harbor/00000738.jpg 52
train/b/bedroom/00000332.jpg 18


In [1]:
#Data Parameters....
CROP_SIZE = 98
BATCH_SIZE = 100
IMAGE_ROOT = "data/images/" #dont change this
BIAS_CONSTANT = 0
NET_PATH = "train-net.prototxt"
VAL_NET_PATH = "test-net.prototxt"
SOLVER_PATH = "solver.prototxt"

#Learning Parameters
weight_param = dict(lr_mult=1, decay_mult=1)
bias_param   = dict(lr_mult=2, decay_mult=0)
learned_param = [weight_param, bias_param]
conv_filler = dict(type='xavier')
const_filler = dict(type='constant', value=BIAS_CONSTANT)
fc_filler = dict(type='xavier')

In [2]:
def get_data_layer(source,train=True):
    mean = [104, 117, 123]  # per-channel mean of the BGR image pixels
    transform_param = dict(mirror=train, crop_size=CROP_SIZE, mean_value=mean)
    places_data, places_labels = L.ImageData(transform_param=transform_param,
        source=source, root_folder=IMAGE_ROOT, shuffle=train,
        batch_size=BATCH_SIZE, ntop=2)
    return places_data,places_labels

train_data,train_labels = get_data_layer(train_file_path,True)
val_data,val_labels = get_data_layer(val_file_path,False)

NameError: name 'train_file_path' is not defined

In [3]:
def conv_relu(bottom, ks, nout, stride=1, pad=0, group=1,weight_filler=conv_filler, bias_filler=const_filler):
    conv = L.Convolution(bottom, kernel_size=ks, stride=stride,
                         num_output=nout, pad=pad, group=group, param=learned_param,
                         weight_filler=weight_filler, bias_filler=bias_filler)
    relu = L.ReLU(conv, in_place=True)
    return conv, relu

In [60]:
def fc_relu(bottom, nout,weight_filler=fc_filler, bias_filler=const_filler):
    fc = L.InnerProduct(bottom, num_output=nout, param=learned_param,weight_filler=weight_filler, bias_filler=bias_filler)
    relu = L.ReLU(fc, in_place=True)
    return fc, relu

In [61]:
def max_pool(bottom, ks, stride=1):
    return L.Pooling(bottom, pool=P.Pooling.MAX, kernel_size=ks, stride=stride)

In [62]:
def places_net(data, labels, train=False,num_classes=100):
    n = caffe.NetSpec()
    n.data = data
    n.conv1, n.relu1 = conv_relu(n.data, 11, 96, stride=4)
    n.pool1 = max_pool(n.relu1, 3, stride=2)
    n.conv2, n.relu2 = conv_relu(n.pool1, 5, 256, pad=2, group=2)
    n.pool2 = max_pool(n.relu2, 3, stride=2)
    n.conv3, n.relu3 = conv_relu(n.pool2, 3, 384, pad=1)
    n.conv4, n.relu4 = conv_relu(n.relu3, 3, 384, pad=1, group=2)
    n.conv5, n.relu5 = conv_relu(n.relu4, 3, 256, pad=1, group=2)
    n.pool5 = max_pool(n.relu5, 3, stride=2)
    n.fc6, n.relu6 = fc_relu(n.pool5, 1024)
    n.drop6 = L.Dropout(n.relu6, in_place=True)
    n.fc7, n.relu7 = fc_relu(n.drop6, 1024)
    n.drop7 = L.Dropout(n.relu7, in_place=True)
    preds = n.fc8 = L.InnerProduct(n.drop7, num_output=num_classes, param=learned_param)
    if not train:
        # Compute the per-label probabilities at test/inference time.
        preds = n.probs = L.Softmax(n.fc8)
    n.label = labels
    n.loss = L.SoftmaxWithLoss(n.fc8, n.label)
    n.accuracy_at_1 = L.Accuracy(preds, n.label)
    n.accuracy_at_5 = L.Accuracy(preds, n.label,accuracy_param=dict(top_k=5))
    if train:
        net_path = NET_PATH
    else:
        net_paht = 
    with open(net_path,'w') as f:
        f.write(str(n.to_proto()))
    return net_path

In [63]:
train_net_file = places_net(train_data,train_labels,train=True)
val_net_file = places_net(val_data,val_labels,train=False)

In [66]:
def places_solver():
    s = caffe_pb2.SolverParameter()
    s.train_net = train_net_file
    s.test_net.append(val_net_file)
    
    s.test_interval = 1000
    s.test_iter.append(100)
    s.iter_size = 1

    # Solve using the stochastic gradient descent (SGD) algorithm.
    # Other choices include 'Adam' and 'RMSProp'.
    #s.type = 'SGD'

    # The following settings (base_lr, lr_policy, gamma, stepsize, and max_iter),
    # define the following learning rate schedule:
    #   Iterations [  0, 20K) -> learning rate 0.01   = base_lr
    #   Iterations [20K, 40K) -> learning rate 0.001  = base_lr * gamma
    #   Iterations [40K, 50K) -> learning rate 0.0001 = base_lr * gamma^2
    s.base_lr = 0.01
    s.lr_policy = 'step'
    s.gamma =  0.1
    s.stepsize = 100000
    s.max_iter = 450000

    # Set other SGD hyperparameters. 
    #`momentum` = takes a weighted average of the current gradient and previous gradients -> more stable learning
    # weight decay = regularizes learning; prevents overfitting
    s.momentum = 0.9
    s.weight_decay = 0.0005

    # Display the current training loss and accuracy every `display` iterations.
    s.display = 100
    s.average_loss = 10
    # Seed the RNG for deterministic results.
    s.random_seed = 1

    # Snapshots are files used to store networks we've trained
    s.snapshot = s.stepsize // 2
    s.snapshot_prefix = "snapshot/"
    if not os.path.exists(s.snapshot_prefix):
        os.makedirs(s.snapshot_prefix)
    with open(SOLVER_PATH,'w') as f:
        f.write(str(s))
    return SOLVER_PATH  

In [69]:
solver_file = places_solver()
solver = caffe.SGDSolver(solver_file) #change the invocation method to change solver

In [71]:
def train_net(iters,disp):
    outputs = sorted(solver.net.outputs)
    def str_output(output):
        value = solver.net.blobs[output].data
        if output.startswith('accuracy'):
            valstr = '%5.2f%%' % (100 * value, )
        else:
            valstr = '%6f' % value
        return '%s = %s' % (output, valstr)
    def disp_outputs(iteration, iter_pad_len=len(str(iters))):
        metrics = '; '.join(str_output(o) for o in outputs)
        return 'Iteration %*d: %s' % (iter_pad_len, iteration, metrics)
    # We could just call `solver.solve()` rather than `step()`ing in a loop.
    # (If we hadn't set GLOG_minloglevel = 3 at the top of this file, Caffe
    # would display loss/accuracy information during training.)
    previous_time = None
    for iteration in xrange(iters):
        solver.step(1)
        if (disp > 0) and (iteration % disp == 0):
            current_time = time.clock()
            if previous_time is None:
                benchmark = ''
            else:
                time_per_iter = (current_time - previous_time) / disp
                benchmark = ' (%5f s/it)' % time_per_iter
            previous_time = current_time
            print disp_outputs(iteration), benchmark
    # Print accuracy for last iteration.
    solver.net.forward()
    disp_outputs(iters)
    #solver.net.save(snapshot_at_iteration(args.iters))

In [ ]:
train_net(500,100)